In [16]:
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import tweepy
import pandas as pd
import numpy as np
import seaborn as sns

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JCharles\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JCharles\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
#En esta parte solo se utilizan las llaves de la API para accceder a los datos de Twitter.

consumer_key = "3jrzOdnUpv9A6I9zdObjpSafU"

consumer_secret = "d0c4HhVMruXgjRGLbdS1mjsepjs9PWCygXYycnFA3Olu1PVIH2" 

access_token = "73235731-JOFtWxEt6r5ecrOw0xQIJ6BNgl7FX2OiupGybuHfq"

access_token_secret = "7H8EVyhQB1zpKut71HbQn2BYshDQTuNjaOnS083o6CKQ0"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


In [18]:
#crear dataframe de cuentas de twitter

accounts = pd.read_csv("accounts.csv")
accounts.head()

,Account
0,realDonaldTrump
1,haaretzcom
2,FT
3,ONU_es
4,CNBC


In [24]:
#Dataframe de acciones
actions = pd.read_csv("actions.csv")
actions.head()

,action
0,$FB
1,$AMZN
2,$NFLX
3,$GOOGL
4,$AAPL


In [ ]:
#crear data frame
tweets = api.user_timeline(screen_name="taikikawa", count=200)
    
def tweets_to_data_frame(tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])
        df['user']= np.array(tweet.screen_name for tweet in tweets)
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df
    
tweets_to_data_frame(tweets)    

In [25]:
#imprimir tweets

public_tweets = api.search('$AAPL')     #Elige el tema a imprimir

for tweet in public_tweets:
    print(tweet.text)          #Imprime los tweets
    analysis = TextBlob(tweet.text) 
    print(analysis.sentiment) #imprime el analisis de sentimiento
    print("")
    


Join Binance for Bonus on trading crypto !

Click here: https://t.co/ggzQI2hlqr   

$NXC $LEND $BITB $XDN $BTCD… https://t.co/frzBPzgwY6
Sentiment(polarity=0.0, subjectivity=0.0)

Buy/Sell Bitcoin changes with up to 100x Leverage at Bitmex! 💘😱

Get a 10% Fee Discount:

Click here:… https://t.co/yhExbjrOgY
Sentiment(polarity=0.0, subjectivity=0.0)

BitMEX lets you leverage trade the top crypto coins !

Click here: https://t.co/qixymNRS7n   

$XRP $XLM $EOS $BCH… https://t.co/7UgGRdagJZ
Sentiment(polarity=0.625, subjectivity=0.5)

Join Binance for Bonus on trading crypto !

Click here: https://t.co/actknETvMV   

$NXC $LEND $BITB $XDN $BTCD… https://t.co/tGYLoaoPEx
Sentiment(polarity=0.0, subjectivity=0.0)

BitMEX lets you leverage trade the top crypto coins !

Click here: https://t.co/HyF12ciuc7   

$XRP $XLM $EOS $BCH… https://t.co/tG1MV3J8SS
Sentiment(polarity=0.625, subjectivity=0.5)

Binance gives free money through airdrops !

Click here: https://t.co/x5DABHRqqY    

$BTC $ETH $XRP